# Cargar el dataset estratificado gracias a los archivos CSV

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

# Cargar los CSVs con las rutas absolutas de las imágenes y las etiquetas
train_df = pd.read_csv(r"C:\BCN20000\train_split.csv")
val_df = pd.read_csv(r"C:\BCN20000\val_split.csv")
test_df = pd.read_csv(r"C:\BCN20000\test_split.csv")

# Visualizar las primeras filas de los CSVs cargados
print("Primeras filas del dataset de entrenamiento:")
print(train_df.head())

print("\nPrimeras filas del dataset de validación:")
print(val_df.head())

print("\nPrimeras filas del dataset de prueba:")
print(test_df.head())


Primeras filas del dataset de entrenamiento:
         bcn_filename  age_approx anatom_site_general diagnosis    lesion_id  \
0  BCN_0000003955.jpg        45.0      anterior torso        NV  BCN_0001511   
1  BCN_0000018219.jpg        35.0     lower extremity        NV  BCN_0000149   
2  BCN_0000002541.jpg        70.0      anterior torso       SCC  BCN_0001083   
3  BCN_0000015263.jpg        75.0     upper extremity       MEL  BCN_0003910   
4  BCN_0000005744.jpg        45.0      anterior torso        NV  BCN_0002731   

  capture_date     sex  split  \
0   2015-11-27  female  train   
1   2015-07-03  female  train   
2   2013-04-02    male  train   
3   2015-01-30  female  train   
4   2014-02-13    male  train   

                                            filepath  label  
0  C:\BCN20000\BCN_20k_train\bcn_20k_train\BCN_00...      5  
1  C:\BCN20000\BCN_20k_train\bcn_20k_train\BCN_00...      5  
2  C:\BCN20000\BCN_20k_train\bcn_20k_train\BCN_00...      6  
3  C:\BCN20000\BCN_20k_trai

Procesamiento y normalización de imágenes

In [2]:
# Función para cargar y preprocesar las imágenes
def load_image(image_path):
    image = tf.io.read_file(image_path)  # Leer la imagen desde la ruta
    image = tf.image.decode_jpeg(image, channels=3)  # Decodificar imagen JPG
    image = tf.image.resize(image, [224, 224])  # Redimensionar a un tamaño común (224x224)
    image = tf.cast(image, tf.float32) / 255.0  # Normalizar las imágenes a [0, 1]
    return image

# Función para preparar los datos de entrada
def prepare_data(df):
    # Cargar imágenes y etiquetas desde las columnas 'filepath' y 'label'
    image_paths = df['filepath'].values
    labels = df['label'].values
    
    # Convertir las rutas y etiquetas a tensores de TensorFlow
    image_paths = tf.constant(image_paths)
    labels = tf.constant(labels)
    
    # Cargar imágenes y preprocesarlas
    images = tf.map_fn(load_image, image_paths, dtype=tf.float32)
    
    return tf.data.Dataset.from_tensor_slices((images, labels))  # Crear datasets

In [3]:
# Balanceo de clases utilizando class_weight
class_weights = {0: 1.0, 1: 2.0, 2: 3.0, 3: 1.5, 4: 1.0, 5: 2.5, 6: 3.0, 7: 1.5}

Crear el dataset de entrenamiento, validación y prueba.
Usamos la función prepare_data para crear datasets para el entrenamiento, validación y prueba, y luego los preparamos para el entrenamiento utilizando batch y shuffle.

In [4]:
# Preparar los datasets con balanceo
train_dataset = prepare_data(train_df).shuffle(1000).batch(32)  # Barajar y agrupar en lotes
val_dataset = prepare_data(val_df).batch(32)  # Validación sin barajar
test_dataset = prepare_data(test_df).batch(32)  # Test sin barajar

# Ver el tamaño de los datasets
print("Tamaño del dataset de entrenamiento:", len(train_df))
print("Tamaño del dataset de validación:", len(val_df))
print("Tamaño del dataset de prueba:", len(test_df))

Tamaño del dataset de entrenamiento: 8971
Tamaño del dataset de validación: 997
Tamaño del dataset de prueba: 2445


Construir el modelo con tensorflow, utilizando como base EfficientNetB0 ya que puede ofrecer mejor eficiencia y precisión que MobileNetV2 en tareas de clasificación médica.

In [5]:
# Crear el modelo EfficientNetB0
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout

base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [6]:
# Congelar las capas del modelo base para que no se entrenen
base_model.trainable = False


In [7]:
# Crear el modelo con EfficientNetB0 como base
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dense(8, activation='softmax')  # Ajusta el número de clases según tu dataset
])

In [8]:
# Compilar el modelo
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='sparse_categorical_crossentropy',  # Utilizamos sparse_categorical_crossentropy para etiquetas enteras
              metrics=['accuracy'])

In [9]:
# Resumen del modelo
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb0 (Functional)  (None, 7, 7, 1280)       4049571   
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 128)               163968    
                                                                 
 dense_1 (Dense)             (None, 8)                 1032      
                                                                 
Total params: 4,214,571
Trainable params: 165,000
Non-trainable params: 4,049,571
_________________________________________________________________


In [10]:
# Entrenar el modelo
history = model.fit(
    train_dataset,
    epochs=130,  # Entrenar durante 130 épocas (como en el código de PyTorch)
    validation_data=val_dataset,
    verbose=1,
    class_weight=class_weights  # Ajustar pesos de las clases
)

Epoch 1/130
281/281 [==============================] - 334s 1s/step - loss: 3.1652 - accuracy: 0.3411 - val_loss: 1.6905 - val_accuracy: 0.3420
Epoch 2/130
281/281 [==============================] - 310s 1s/step - loss: 3.1065 - accuracy: 0.3421 - val_loss: 1.6837 - val_accuracy: 0.3420
Epoch 3/130
281/281 [==============================] - 328s 1s/step - loss: 3.1071 - accuracy: 0.3421 - val_loss: 1.6917 - val_accuracy: 0.3420
Epoch 4/130
281/281 [==============================] - 327s 1s/step - loss: 3.1061 - accuracy: 0.3421 - val_loss: 1.6958 - val_accuracy: 0.3420
Epoch 5/130
 76/281 [=======>......................] - ETA: 3:31 - loss: 3.1092 - accuracy: 0.3561

KeyboardInterrupt: 

Graficar precisión y pérdida

In [ ]:
import matplotlib.pyplot as plt

# Graficar precisión
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Precisión entrenamiento')
plt.plot(history.history['val_accuracy'], label='Precisión validación')
plt.title('Precisión durante el entrenamiento')
plt.xlabel('Época')
plt.ylabel('Precisión')
plt.legend()

# Graficar pérdida
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Pérdida entrenamiento')
plt.plot(history.history['val_loss'], label='Pérdida validación')
plt.title('Pérdida durante el entrenamiento')
plt.xlabel('Época')
plt.ylabel('Pérdida')
plt.legend()

plt.tight_layout()
plt.show()

Evaluar el modelo en el conjunto de prueba. Una vez entrenado el modelo, evaluamos su desempeño en el conjunto de prueba para verificar su precisión.

In [ ]:
# Evaluar el modelo en el conjunto de prueba
test_loss, test_acc = model.evaluate(test_dataset, verbose=1)
print(f"Test accuracy: {test_acc}")

# Realizar predicciones

In [ ]:
# Función para predecir una imagen
def predict_image(image_path):
    image = load_image(image_path)
    image = tf.expand_dims(image, axis=0)  # Añadir una dimensión para el batch
    prediction = model.predict(image)
    predicted_class = np.argmax(prediction, axis=-1)  # Obtener la clase con mayor probabilidad
    return predicted_class

# Ejemplo de predicción
image_path = 'ruta/a/una/imagen/nueva.jpg'
predicted_class = predict_image(image_path)
print(f"Predicción de clase: {predicted_class}")